In [44]:

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
#import xgboost as xgb
sns.set_style('whitegrid') 
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import pipeline, grid_search

import random
random.seed(2016)

import time

In [52]:
def main(input1='../data/preprocessing_train_df.csv', input2='../data/preprocessing_test_df.csv'):



    start_time = time.time() 
    
#load preprocessed training data as a dataframe
    train_df = pd.read_csv(input1, index_col=0)
    test_df  = pd.read_csv(input2, index_col=0)

#  test_df = pd.read_csv("../data/preprocessing_test_df.csv" , index_col=0)
#  train_df.head()
#    train_df.info()
#    test_df.info()
#    test_df.head(3)


    x_train = train_df.iloc[:, 2:]
    y_train = train_df.iloc[:, 1]

    id_test = test_df['PassengerId']
    x_test = test_df.iloc[:,1:]
#    x_train.info()
    
    
    print('--- Features Set: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
    print('Number of Features: ', len(x_train.columns.tolist()))
    
    
# Grid Search for RandomForestClassifier    
    
    
    rf = RandomForestClassifier(oob_score=True, random_state=1, n_jobs=1)

    param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10], 
              "min_samples_split" : [2, 4, 10, 12, 16], "n_estimators": [50, 100, 400, 700, 1000],
             "max_features":[7,8,10,14,18,20,25,30]}


#    param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [ 10], 
#              "min_samples_split" : [ 10], "n_estimators": [100],
#             "max_features":[8,10,14,18, 20, 25, 30]}


    model = grid_search.GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=1)
    model.fit(x_train, y_train)
    
    
    print('--- Grid Search Completed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
    print('Best Params:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)

    
    y_pred = model.predict(x_test)

    pd.DataFrame({'PassengerId': id_test, 'Survived': y_pred}).to_csv('../data/submission_rf.csv', index=False)
    
    print('--- Submission Generated: %s minutes ---' % round(((time.time() - start_time) / 60), 2))


#if __name__ == '__main__':


main()


--- Features Set: 0.0 minutes ---
('Number of Features: ', 44)
--- Grid Search Completed: 249.63 minutes ---
Best Params:
{'max_features': 30, 'min_samples_split': 10, 'n_estimators': 50, 'criterion': 'entropy', 'min_samples_leaf': 1}
Best CV Score:
-0.850729517396
--- Submission Generated: 249.63 minutes ---
